# 第5章：Spark Streaming 實戰指南

本章節將深入學習 Spark Streaming，包括結構化串流、實時數據處理、視窗操作等。

## 學習目標
- 理解 Spark Streaming 的核心概念
- 掌握結構化串流 (Structured Streaming) 的使用
- 學習實時數據處理和視窗操作
- 實戰各種數據源的串流處理

## 1. 環境設置

In [ ]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
import time
import json
import threading
import random
from datetime import datetime, timedelta

In [ ]:
# 創建 SparkSession
spark = SparkSession.builder \
    .appName("SparkStreaming學習") \
    .master("local[*]") \
    .config("spark.sql.adaptive.enabled", "true") \
    .config("spark.sql.streaming.checkpointLocation", "/tmp/spark-checkpoints") \
    .getOrCreate()

# 設置日志級別
spark.sparkContext.setLogLevel("WARN")

print(f"Spark版本: {spark.version}")
print(f"Spark UI: http://localhost:4040")

## 2. 結構化串流基礎

In [ ]:
# 創建模擬數據生成器
import os
import tempfile
import shutil

# 創建臨時目錄
temp_dir = tempfile.mkdtemp()
input_dir = os.path.join(temp_dir, "input")
output_dir = os.path.join(temp_dir, "output")
checkpoint_dir = os.path.join(temp_dir, "checkpoint")

os.makedirs(input_dir, exist_ok=True)
os.makedirs(output_dir, exist_ok=True)
os.makedirs(checkpoint_dir, exist_ok=True)

print(f"臨時目錄: {temp_dir}")
print(f"輸入目錄: {input_dir}")
print(f"輸出目錄: {output_dir}")
print(f"檢查點目錄: {checkpoint_dir}")

## 3. 文件串流處理

In [ ]:
# 定義數據模式
schema = StructType([
    StructField("timestamp", StringType(), True),
    StructField("user_id", StringType(), True),
    StructField("event_type", StringType(), True),
    StructField("value", IntegerType(), True)
])

# 創建文件串流
file_stream = spark.readStream \
    .format("json") \
    .schema(schema) \
    .option("path", input_dir) \
    .load()

print("文件串流創建完成")
print("Schema:")
file_stream.printSchema()
print(f"是否為串流: {file_stream.isStreaming}")

In [ ]:
# 創建數據生成器函數
def generate_sample_data(filename, num_records=10):
    """生成樣本數據"""
    data = []
    event_types = ['click', 'view', 'purchase', 'login']
    user_ids = [f'user_{i}' for i in range(1, 21)]
    
    for i in range(num_records):
        record = {
            "timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
            "user_id": random.choice(user_ids),
            "event_type": random.choice(event_types),
            "value": random.randint(1, 100)
        }
        data.append(record)
    
    # 寫入文件
    filepath = os.path.join(input_dir, filename)
    with open(filepath, 'w') as f:
        for record in data:
            f.write(json.dumps(record) + '\n')
    
    print(f"生成 {num_records} 條記錄到 {filename}")
    return len(data)

# 生成初始數據
generate_sample_data("initial_data.json", 20)

In [ ]:
# 基本串流處理
basic_query = file_stream.writeStream \
    .outputMode("append") \
    .format("console") \
    .trigger(processingTime='10 seconds') \
    .option("truncate", "false") \
    .start()

print("基本串流處理已啟動")
print(f"查詢 ID: {basic_query.id}")
print(f"查詢狀態: {basic_query.status}")

# 等待一段時間
time.sleep(15)

# 添加更多數據
generate_sample_data("batch_2.json", 10)

# 再等待一段時間
time.sleep(10)

# 停止查詢
basic_query.stop()
print("基本串流處理已停止")

## 4. 聚合和轉換

In [ ]:
# 聚合查詢
aggregated_stream = file_stream.groupBy("event_type") \
    .agg(
        count("*").alias("event_count"),
        avg("value").alias("avg_value"),
        max("value").alias("max_value"),
        min("value").alias("min_value")
    )

# 啟動聚合查詢
agg_query = aggregated_stream.writeStream \
    .outputMode("complete") \
    .format("console") \
    .trigger(processingTime='10 seconds') \
    .option("truncate", "false") \
    .start()

print("聚合查詢已啟動")

# 連續添加數據
for i in range(3):
    time.sleep(5)
    generate_sample_data(f"batch_{i+3}.json", 15)
    print(f"添加了第 {i+3} 批數據")

# 等待處理完成
time.sleep(15)

# 停止查詢
agg_query.stop()
print("聚合查詢已停止")

## 5. 視窗操作

In [ ]:
# 時間視窗聚合
windowed_stream = file_stream \
    .withColumn("timestamp", to_timestamp(col("timestamp"), "yyyy-MM-dd HH:mm:ss")) \
    .withWatermark("timestamp", "10 seconds") \
    .groupBy(
        window(col("timestamp"), "30 seconds", "10 seconds"),
        col("event_type")
    ) \
    .agg(
        count("*").alias("event_count"),
        avg("value").alias("avg_value")
    ) \
    .select(
        col("window.start").alias("window_start"),
        col("window.end").alias("window_end"),
        col("event_type"),
        col("event_count"),
        round(col("avg_value"), 2).alias("avg_value")
    )

# 啟動視窗查詢
window_query = windowed_stream.writeStream \
    .outputMode("update") \
    .format("console") \
    .trigger(processingTime='5 seconds') \
    .option("truncate", "false") \
    .start()

print("視窗查詢已啟動")

# 連續添加數據測試視窗
for i in range(4):
    time.sleep(8)
    generate_sample_data(f"window_batch_{i+1}.json", 12)
    print(f"添加了視窗測試數據 {i+1}")

# 等待處理完成
time.sleep(20)

# 停止查詢
window_query.stop()
print("視窗查詢已停止")

## 6. 過濾和轉換

In [ ]:
# 複雜轉換和過濾
transformed_stream = file_stream \
    .withColumn("timestamp", to_timestamp(col("timestamp"), "yyyy-MM-dd HH:mm:ss")) \
    .withColumn("hour", hour(col("timestamp"))) \
    .withColumn("value_category", 
                when(col("value") >= 80, "high")
                .when(col("value") >= 50, "medium")
                .otherwise("low")) \
    .withColumn("is_high_value", col("value") >= 75) \
    .filter(col("event_type").isin(["click", "purchase"])) \
    .select(
        col("timestamp"),
        col("user_id"),
        col("event_type"),
        col("value"),
        col("hour"),
        col("value_category"),
        col("is_high_value")
    )

# 啟動轉換查詢
transform_query = transformed_stream.writeStream \
    .outputMode("append") \
    .format("console") \
    .trigger(processingTime='8 seconds') \
    .option("truncate", "false") \
    .start()

print("轉換查詢已啟動")

# 添加測試數據
for i in range(3):
    time.sleep(6)
    generate_sample_data(f"transform_batch_{i+1}.json", 8)
    print(f"添加了轉換測試數據 {i+1}")

# 等待處理完成
time.sleep(15)

# 停止查詢
transform_query.stop()
print("轉換查詢已停止")

## 7. 多個輸出目標

In [ ]:
# 創建多個查詢同時處理
# 查詢1：保存到文件
file_output_query = file_stream.writeStream \
    .outputMode("append") \
    .format("json") \
    .option("path", output_dir) \
    .option("checkpointLocation", checkpoint_dir + "/file_output") \
    .trigger(processingTime='10 seconds') \
    .start()

# 查詢2：實時統計
stats_query = file_stream.groupBy("event_type", "user_id") \
    .agg(
        count("*").alias("event_count"),
        sum("value").alias("total_value")
    ) \
    .writeStream \
    .outputMode("complete") \
    .format("console") \
    .trigger(processingTime='10 seconds') \
    .start()

print("多個查詢已啟動")
print(f"文件輸出查詢 ID: {file_output_query.id}")
print(f"統計查詢 ID: {stats_query.id}")

# 添加數據
for i in range(3):
    time.sleep(8)
    generate_sample_data(f"multi_output_{i+1}.json", 10)
    print(f"添加了多輸出測試數據 {i+1}")

# 等待處理完成
time.sleep(15)

# 停止所有查詢
file_output_query.stop()
stats_query.stop()
print("所有查詢已停止")

# 檢查輸出文件
output_files = os.listdir(output_dir)
print(f"\n輸出文件: {output_files}")

## 8. 狀態管理和容錯

In [ ]:
# 帶狀態的查詢
stateful_stream = file_stream \
    .groupBy("user_id") \
    .agg(
        count("*").alias("total_events"),
        sum("value").alias("total_value"),
        avg("value").alias("avg_value"),
        collect_list("event_type").alias("event_types")
    ) \
    .withColumn("avg_value", round(col("avg_value"), 2))

# 啟動狀態查詢
stateful_query = stateful_stream.writeStream \
    .outputMode("complete") \
    .format("console") \
    .option("checkpointLocation", checkpoint_dir + "/stateful") \
    .trigger(processingTime='10 seconds') \
    .option("truncate", "false") \
    .start()

print("狀態查詢已啟動")

# 模擬數據添加
for i in range(4):
    time.sleep(8)
    generate_sample_data(f"stateful_batch_{i+1}.json", 6)
    print(f"添加了狀態測試數據 {i+1}")

# 等待處理完成
time.sleep(15)

# 查看查詢統計
print("\n查詢進度:")
print(stateful_query.lastProgress)

# 停止查詢
stateful_query.stop()
print("狀態查詢已停止")

## 9. 實時數據品質監控

In [ ]:
# 數據品質監控
quality_stream = file_stream \
    .withColumn("timestamp", to_timestamp(col("timestamp"), "yyyy-MM-dd HH:mm:ss")) \
    .withColumn("is_valid_timestamp", col("timestamp").isNotNull()) \
    .withColumn("is_valid_user", col("user_id").isNotNull() & (length(col("user_id")) > 0)) \
    .withColumn("is_valid_event", col("event_type").isin(["click", "view", "purchase", "login"])) \
    .withColumn("is_valid_value", (col("value") >= 0) & (col("value") <= 100)) \
    .withColumn("is_valid_record", 
                col("is_valid_timestamp") & 
                col("is_valid_user") & 
                col("is_valid_event") & 
                col("is_valid_value"))

# 品質統計
quality_stats = quality_stream.agg(
    count("*").alias("total_records"),
    sum(when(col("is_valid_record"), 1).otherwise(0)).alias("valid_records"),
    sum(when(col("is_valid_timestamp"), 1).otherwise(0)).alias("valid_timestamps"),
    sum(when(col("is_valid_user"), 1).otherwise(0)).alias("valid_users"),
    sum(when(col("is_valid_event"), 1).otherwise(0)).alias("valid_events"),
    sum(when(col("is_valid_value"), 1).otherwise(0)).alias("valid_values")
) \
.withColumn("data_quality_score", 
            round(col("valid_records") / col("total_records") * 100, 2))

# 啟動品質監控
quality_query = quality_stats.writeStream \
    .outputMode("complete") \
    .format("console") \
    .trigger(processingTime='10 seconds') \
    .option("truncate", "false") \
    .start()

print("數據品質監控已啟動")

# 添加測試數據（包含一些無效數據）
def generate_quality_test_data(filename, num_records=10):
    """生成包含品質問題的測試數據"""
    data = []
    event_types = ['click', 'view', 'purchase', 'login', 'invalid_event']  # 包含無效事件
    user_ids = [f'user_{i}' for i in range(1, 16)] + ['', None]  # 包含無效用戶
    
    for i in range(num_records):
        # 隨機生成一些無效數據
        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S") if random.random() > 0.1 else None
        user_id = random.choice(user_ids)
        event_type = random.choice(event_types)
        value = random.randint(-10, 110)  # 包含超出範圍的值
        
        record = {
            "timestamp": timestamp,
            "user_id": user_id,
            "event_type": event_type,
            "value": value
        }
        data.append(record)
    
    # 寫入文件
    filepath = os.path.join(input_dir, filename)
    with open(filepath, 'w') as f:
        for record in data:
            f.write(json.dumps(record) + '\n')
    
    print(f"生成 {num_records} 條品質測試記錄到 {filename}")
    return len(data)

# 添加品質測試數據
for i in range(3):
    time.sleep(8)
    generate_quality_test_data(f"quality_test_{i+1}.json", 12)
    print(f"添加了品質測試數據 {i+1}")

# 等待處理完成
time.sleep(15)

# 停止查詢
quality_query.stop()
print("數據品質監控已停止")

## 10. 實時告警系統

In [ ]:
# 創建告警邏輯
def create_alert_stream():
    """創建實時告警流"""
    # 高價值事件告警
    high_value_alerts = file_stream \
        .filter(col("value") >= 90) \
        .withColumn("alert_type", lit("HIGH_VALUE_EVENT")) \
        .withColumn("alert_message", 
                   concat(lit("High value event detected: "), 
                         col("event_type"), lit(" with value "), col("value")))
    
    # 頻繁用戶活動告警
    frequent_user_alerts = file_stream \
        .withColumn("timestamp", to_timestamp(col("timestamp"), "yyyy-MM-dd HH:mm:ss")) \
        .withWatermark("timestamp", "30 seconds") \
        .groupBy(
            window(col("timestamp"), "1 minute"),
            col("user_id")
        ) \
        .count() \
        .filter(col("count") >= 5) \
        .withColumn("alert_type", lit("FREQUENT_USER_ACTIVITY")) \
        .withColumn("alert_message", 
                   concat(lit("Frequent activity detected for user "), 
                         col("user_id"), lit(": "), col("count"), lit(" events in 1 minute")))
    
    return high_value_alerts, frequent_user_alerts

# 創建告警流
high_value_alerts, frequent_user_alerts = create_alert_stream()

# 啟動高價值告警
high_value_query = high_value_alerts.select(
    col("timestamp"),
    col("user_id"),
    col("event_type"),
    col("value"),
    col("alert_type"),
    col("alert_message")
).writeStream \
    .outputMode("append") \
    .format("console") \
    .trigger(processingTime='5 seconds') \
    .option("truncate", "false") \
    .start()

print("告警系統已啟動")

# 生成觸發告警的數據
def generate_alert_data(filename, num_records=10):
    """生成會觸發告警的數據"""
    data = []
    event_types = ['click', 'view', 'purchase', 'login']
    
    # 生成高價值事件
    for i in range(num_records // 2):
        record = {
            "timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
            "user_id": f'user_{random.randint(1, 5)}',
            "event_type": random.choice(event_types),
            "value": random.randint(85, 100)  # 高價值
        }
        data.append(record)
    
    # 生成頻繁用戶活動
    frequent_user = 'user_frequent'
    for i in range(num_records // 2):
        record = {
            "timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
            "user_id": frequent_user,
            "event_type": random.choice(event_types),
            "value": random.randint(30, 70)
        }
        data.append(record)
    
    # 寫入文件
    filepath = os.path.join(input_dir, filename)
    with open(filepath, 'w') as f:
        for record in data:
            f.write(json.dumps(record) + '\n')
    
    print(f"生成 {num_records} 條告警測試記錄到 {filename}")
    return len(data)

# 添加告警測試數據
for i in range(3):
    time.sleep(6)
    generate_alert_data(f"alert_test_{i+1}.json", 8)
    print(f"添加了告警測試數據 {i+1}")

# 等待處理完成
time.sleep(15)

# 停止查詢
high_value_query.stop()
print("告警系統已停止")

## 11. 查詢管理和監控

In [ ]:
# 創建監控查詢
monitoring_stream = file_stream.agg(
    count("*").alias("total_records"),
    countDistinct("user_id").alias("unique_users"),
    countDistinct("event_type").alias("unique_events"),
    avg("value").alias("avg_value"),
    min("value").alias("min_value"),
    max("value").alias("max_value")
) \
.withColumn("avg_value", round(col("avg_value"), 2)) \
.withColumn("processing_time", current_timestamp())

# 啟動監控查詢
monitoring_query = monitoring_stream.writeStream \
    .outputMode("complete") \
    .format("console") \
    .trigger(processingTime='10 seconds') \
    .option("truncate", "false") \
    .start()

print("監控查詢已啟動")
print(f"查詢 ID: {monitoring_query.id}")
print(f"查詢名稱: {monitoring_query.name}")

# 添加監控測試數據
for i in range(3):
    time.sleep(8)
    generate_sample_data(f"monitoring_test_{i+1}.json", 15)
    
    # 顯示查詢狀態
    print(f"\n=== 查詢狀態 {i+1} ===")
    print(f"查詢狀態: {monitoring_query.status}")
    print(f"最近進度: {monitoring_query.lastProgress}")

# 等待處理完成
time.sleep(15)

# 獲取最終統計
final_progress = monitoring_query.lastProgress
print("\n=== 最終統計 ===")
print(f"處理的批次數: {final_progress.get('batchId', 'N/A')}")
print(f"輸入行數: {final_progress.get('inputRowsPerSecond', 'N/A')}")
print(f"處理行數: {final_progress.get('processedRowsPerSecond', 'N/A')}")

# 停止查詢
monitoring_query.stop()
print("監控查詢已停止")

## 12. 自定義輸出接收器

In [ ]:
# 自定義輸出函數
def custom_output_function(df, epoch_id):
    """自定義輸出處理函數"""
    print(f"\n=== 處理批次 {epoch_id} ===")
    
    # 獲取基本統計
    count = df.count()
    print(f"記錄數: {count}")
    
    if count > 0:
        # 顯示樣本數據
        print("樣本數據:")
        df.show(5, truncate=False)
        
        # 按事件類型統計
        event_stats = df.groupBy("event_type").count().collect()
        print("事件類型統計:")
        for row in event_stats:
            print(f"  {row.event_type}: {row.count}")
        
        # 可以在這裡添加更多自定義處理邏輯
        # 例如：發送通知、寫入數據庫、調用API等
    
    print(f"批次 {epoch_id} 處理完成")

# 使用自定義輸出
custom_query = file_stream.writeStream \
    .foreachBatch(custom_output_function) \
    .trigger(processingTime='10 seconds') \
    .start()

print("自定義輸出查詢已啟動")

# 添加測試數據
for i in range(3):
    time.sleep(8)
    generate_sample_data(f"custom_output_{i+1}.json", 8)
    print(f"添加了自定義輸出測試數據 {i+1}")

# 等待處理完成
time.sleep(15)

# 停止查詢
custom_query.stop()
print("自定義輸出查詢已停止")

## 13. 性能優化和最佳實踐

In [ ]:
# 性能優化示例
print("=== 性能優化示例 ===")

# 1. 合理的觸發間隔
optimized_stream = file_stream \
    .repartition(4) \
    .cache() \
    .groupBy("event_type") \
    .agg(
        count("*").alias("count"),
        avg("value").alias("avg_value")
    )

# 2. 使用適當的輸出模式
perf_query = optimized_stream.writeStream \
    .outputMode("complete") \
    .format("console") \
    .trigger(processingTime='10 seconds') \
    .option("checkpointLocation", checkpoint_dir + "/performance") \
    .start()

print("性能優化查詢已啟動")

# 監控性能指標
start_time = time.time()

# 添加測試數據
for i in range(3):
    time.sleep(8)
    generate_sample_data(f"perf_test_{i+1}.json", 20)
    
    # 顯示性能指標
    progress = perf_query.lastProgress
    if progress:
        print(f"\n批次 {progress.get('batchId', 'N/A')} 性能指標:")
        print(f"  輸入速率: {progress.get('inputRowsPerSecond', 0):.2f} 行/秒")
        print(f"  處理速率: {progress.get('processedRowsPerSecond', 0):.2f} 行/秒")
        print(f"  批次持續時間: {progress.get('batchDuration', 0)} ms")

# 等待處理完成
time.sleep(15)

# 計算總處理時間
total_time = time.time() - start_time
print(f"\n總處理時間: {total_time:.2f} 秒")

# 停止查詢
perf_query.stop()
print("性能優化查詢已停止")

## 14. 實戰案例：實時儀表板

In [ ]:
# 創建實時儀表板
def create_dashboard():
    """創建實時數據儀表板"""
    
    # 實時統計
    real_time_stats = file_stream \
        .withColumn("timestamp", to_timestamp(col("timestamp"), "yyyy-MM-dd HH:mm:ss")) \
        .withColumn("hour", hour(col("timestamp"))) \
        .groupBy("hour", "event_type") \
        .agg(
            count("*").alias("event_count"),
            avg("value").alias("avg_value"),
            countDistinct("user_id").alias("unique_users")
        ) \
        .withColumn("avg_value", round(col("avg_value"), 2))
    
    # 用戶活動統計
    user_activity = file_stream \
        .groupBy("user_id") \
        .agg(
            count("*").alias("total_events"),
            sum("value").alias("total_value"),
            countDistinct("event_type").alias("unique_events")
        ) \
        .filter(col("total_events") >= 3) \
        .orderBy(col("total_value").desc())
    
    return real_time_stats, user_activity

# 創建儀表板查詢
stats_stream, user_stream = create_dashboard()

# 啟動統計查詢
stats_query = stats_stream.writeStream \
    .outputMode("complete") \
    .format("console") \
    .trigger(processingTime='10 seconds') \
    .option("truncate", "false") \
    .start()

print("實時儀表板已啟動")

# 生成儀表板測試數據
def generate_dashboard_data(filename, num_records=15):
    """生成儀表板測試數據"""
    data = []
    event_types = ['click', 'view', 'purchase', 'login']
    user_ids = [f'user_{i}' for i in range(1, 11)]
    
    current_time = datetime.now()
    
    for i in range(num_records):
        # 生成不同時間的數據
        timestamp = current_time - timedelta(minutes=random.randint(0, 60))
        
        record = {
            "timestamp": timestamp.strftime("%Y-%m-%d %H:%M:%S"),
            "user_id": random.choice(user_ids),
            "event_type": random.choice(event_types),
            "value": random.randint(10, 90)
        }
        data.append(record)
    
    # 寫入文件
    filepath = os.path.join(input_dir, filename)
    with open(filepath, 'w') as f:
        for record in data:
            f.write(json.dumps(record) + '\n')
    
    print(f"生成 {num_records} 條儀表板測試記錄到 {filename}")
    return len(data)

# 添加儀表板測試數據
for i in range(4):
    time.sleep(8)
    generate_dashboard_data(f"dashboard_test_{i+1}.json", 12)
    print(f"添加了儀表板測試數據 {i+1}")

# 等待處理完成
time.sleep(15)

# 停止查詢
stats_query.stop()
print("實時儀表板已停止")

## 15. 總結和清理

In [ ]:
# 清理資源
print("=== Spark Streaming 學習總結 ===")
print("✓ 結構化串流基礎概念")
print("✓ 文件串流處理")
print("✓ 實時聚合和轉換")
print("✓ 視窗操作和水印")
print("✓ 過濾和複雜轉換")
print("✓ 多輸出目標")
print("✓ 狀態管理和容錯")
print("✓ 數據品質監控")
print("✓ 實時告警系統")
print("✓ 查詢管理和監控")
print("✓ 自定義輸出接收器")
print("✓ 性能優化技巧")
print("✓ 實時儀表板實戰")

# 停止所有活動查詢
active_queries = spark.streams.active
print(f"\n活動查詢數: {len(active_queries)}")

for query in active_queries:
    print(f"停止查詢: {query.id}")
    query.stop()

print("所有查詢已停止")

# 清理臨時文件
try:
    shutil.rmtree(temp_dir)
    print(f"已清理臨時目錄: {temp_dir}")
except:
    print(f"無法清理臨時目錄: {temp_dir}")

# 停止 SparkSession
spark.stop()
print("Spark session 已停止")

## 學習重點回顧

### 核心概念
1. **結構化串流**：基於 DataFrame 的串流處理 API
2. **輸出模式**：Append、Complete、Update
3. **觸發器**：控制批次處理頻率
4. **檢查點**：保證容錯和一致性

### 進階特性
1. **視窗操作**：時間視窗聚合
2. **水印**：處理延遲數據
3. **狀態管理**：維護跨批次狀態
4. **自定義輸出**：靈活的輸出處理

### 實戰應用
1. **實時監控**：系統指標、業務指標
2. **數據品質**：實時數據驗證
3. **告警系統**：異常檢測和通知
4. **實時儀表板**：可視化展示

### 最佳實踐
1. **性能優化**：合理分區、緩存策略
2. **容錯機制**：檢查點、重啟策略
3. **監控調試**：查詢狀態、性能指標
4. **資源管理**：內存配置、並行度調整

通過本章學習，您已經掌握了 Spark Streaming 的核心技術，能夠構建高效、可靠的實時數據處理系統。